In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Concatenate
from keras.optimizers import Adam
from keras.utils import to_categorical

# Load the CSV file
df = pd.read_csv('edited_file.csv')

# Assuming the features are in columns 0 to 20, and the labels are in columns 21, 22, and 23
X = df.iloc[:, :21].values
y_emotion = df.iloc[:, 21].values  # Assuming 'emotion' is in column 21
y_age = df.iloc[:, 22].values  # Assuming 'age' is in column 22
y_gender = df.iloc[:, 23].values  # Assuming 'gender' is in column 23

# Use LabelEncoder for 'emotion'
label_encoder = LabelEncoder()
y_emotion_encoded = label_encoder.fit_transform(y_emotion)
num_emotion_classes = len(label_encoder.classes_)

# Convert categorical labels to one-hot encoding
y_emotion = to_categorical(y_emotion_encoded, num_classes=num_emotion_classes)
y_age = to_categorical(y_age, num_classes=8)
y_gender = to_categorical(y_gender, num_classes=2)

# Split the data into training and testing sets
X_train, X_test, y_emotion_train, y_emotion_test, y_age_train, y_age_test, y_gender_train, y_gender_test = train_test_split(
    X, y_emotion, y_age, y_gender, test_size=0.2, random_state=42
)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# CNN Layers
inputs_cnn = Input(shape=(21, 1))
conv1 = Conv1D(32, 3, activation='relu')(inputs_cnn)
pool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(64, 3, activation='relu')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)
flat_cnn = Flatten()(pool2)

# Dense Layers
dense1 = Dense(64, activation='relu')(flat_cnn)
dense2 = Dense(64, activation='relu')(flat_cnn)
dense3 = Dense(64, activation='relu')(flat_cnn)

# Output Layers
output_emotion = Dense(num_emotion_classes, activation='softmax', name='emotion')(dense1)
output_age = Dense(8, activation='softmax', name='age')(dense2)
output_gender = Dense(2, activation='softmax', name='gender')(dense3)

# Create the model
model = Model(inputs=inputs_cnn, outputs=[output_emotion, output_age, output_gender])

# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss={'emotion': 'categorical_crossentropy', 'age': 'categorical_crossentropy', 'gender': 'categorical_crossentropy'},
              metrics={'emotion': 'accuracy', 'age': 'accuracy', 'gender': 'accuracy'})

# Display the model summary
model.summary()

# Train the model
model.fit(X_train, [y_emotion_train, y_age_train, y_gender_train], epochs=10, batch_size=32, validation_split=0.2)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 21, 1)]              0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 19, 32)               128       ['input_1[0][0]']             
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 9, 32)                0         ['conv1d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv1d_1 (Conv1D)           (None, 7, 64)                6208      ['max_pooling1d[0][0]']   

In [4]:
# Evaluate the model on the test set
evaluation = model.evaluate(X_test, [y_emotion_test, y_age_test, y_gender_test])

# Print the evaluation metrics
print(f"Test Loss: {evaluation[0]}")
print(f"Emotion Accuracy: {evaluation[3] * 100:.2f}%")
print(f"Age Accuracy: {evaluation[4] * 100:.2f}%")
print(f"Gender Accuracy: {evaluation[5] * 100:.2f}%")


47/47 [==============================] - 0s 3ms/step - loss: 3.3044 - emotion_loss: 1.4292 - age_loss: 1.3380 - gender_loss: 0.5372 - emotion_accuracy: 0.4177 - age_accuracy: 0.4809 - gender_accuracy: 0.7314
Test Loss: 3.304385185241699
Emotion Accuracy: 53.72%
Age Accuracy: 41.77%
Gender Accuracy: 48.09%


In [8]:
import numpy as np

# Custom input
custom_input = np.array([[2679.9395691628692 ,3347.669488765762, 5745.486745886655
 -625.2181396484375, 111.32093811035156 ,6.3269944190979 ,34.75761413574219,
 31.619901657104492 ,-4.714645862579346 ,-0.486030638217926,
 -4.934024333953857 ,-12.714733123779297 ,-2.0551483631134033,
 -3.7411177158355713, -10.702962875366211, -11.20263671875,
 -12.003522872924805, -8.489580154418945 ,-5.463275909423828,
 -4.954216480255127, -3.715198278427124,-6.338474273681641]])

# Standardize the custom input
custom_input = scaler.transform(custom_input)

# Reshape the input to match the model's input shape
custom_input = custom_input.reshape((1, 21, 1))

# Make predictions
predictions = model.predict(custom_input)

# Extract the predictions for each output
emotion_prediction = predictions[0]
age_prediction = predictions[1]
gender_prediction = predictions[2]

# Decode emotion predictions using the LabelEncoder
emotion_prediction_decoded = label_encoder.inverse_transform(np.argmax(emotion_prediction, axis=1))

# Print the predictions
print(f"Emotion Prediction: {emotion_prediction_decoded}")
print(f"Age Prediction: {np.argmax(age_prediction, axis=1)}")
print(f"Gender Prediction: {np.argmax(gender_prediction, axis=1)}")


1/1 [==============================] - 0s 35ms/step
Emotion Prediction: [0]
Age Prediction: [7]
Gender Prediction: [0]


In [ ]:
# Save the entire model to a HDF5 file
model.save('all_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
